In [ ]:
from huggingface_hub import snapshot_download
from insightface.app import FaceAnalysis
import numpy as np
import cv2
import itertools

c:\Users\nosinska\Desktop\PWr\sem_5\biometry\.venv\Lib\site-packages\albumentations\check_version.py:51: UserWarning: Error fetching version info <urlopen error timed out>
  data = fetch_version_info()


### Pobranie modelu.

Model: https://huggingface.co/fal/AuraFace-v1

In [ ]:
# snapshot_download(
#     "fal/AuraFace-v1",
#     local_dir="models/auraface",
# ) # uncomment for first download
face_app = FaceAnalysis(
    name="auraface",
    providers=["CUDAExecutionProvider", "CPUExecutionProvider"],
    root=".",
)
face_app.prepare(ctx_id=-1)

c:\Users\nosinska\Desktop\PWr\sem_5\biometry\.venv\Lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:115: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: .\models\auraface\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: .\models\auraface\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: .\models\auraface\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: .\models\auraface\glintr100.onnx recognition ['None', 3, 112, 112] 127.5 127.5
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: .\models\auraface\scrfd_10g_bnkps.onnx detection [1, 3, '?', '?'] 127.5 128.0
set det-size: (640, 640)


Funkcje do porównywania wektorów reprezentacji(embeding).

In [2]:
def cosine_similarity(embedding1, embedding2):
    dot_product = np.dot(embedding1, embedding2)
    norm1 = np.linalg.norm(embedding1)
    norm2 = np.linalg.norm(embedding2)
    return dot_product / (norm1 * norm2)

def euclidean_distance(embedding1, embedding2):
    return np.linalg.norm(embedding1 - embedding2)

Generowanie wektora reprezentacji ze zdjęcia.

In [7]:
def get_embedding(image_path):
    input_image = cv2.imread(image_path)

    cv2_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)

    # cv2_image = cv2_image[:, :, ::-1]
    faces = face_app.get(cv2_image)
    embedding = faces[0].normed_embedding
    return embedding

### Część 1 - porównywanie zdjęć różnych osób.

In [ ]:
photos = ["photos/photo-1.jpeg", "photos/photo-2.jpeg", "photos/photo-3.jpeg"]
photo_embd = {}
for photo in photos:
    photo_embd[photo] = get_embedding(photo)
    
pairs = itertools.combinations(photos, 2)

for photo1, photo2 in pairs:
    print(photo1, photo2)
    print(f"Cos: {cosine_similarity(photo_embd[photo1], photo_embd[photo2])}")
    print(f"Euclidean: {euclidean_distance(photo_embd[photo1], photo_embd[photo2])}")

photos/photo-1.jpeg photos/photo-2.jpeg
Cos: 0.030051080510020256
Euclidean: 1.392802119255066
photos/photo-1.jpeg photos/photo-3.jpeg
Cos: 0.12143760919570923
Euclidean: 1.3255659341812134
photos/photo-2.jpeg photos/photo-3.jpeg
Cos: 0.15661245584487915
Euclidean: 1.29875910282135


Początek do takich samych twarzy.

In [15]:
print(f"Cos: {cosine_similarity(get_embedding("photos/bartek-3.jpg"), get_embedding("photos/bartek-2.jpg"))}")
print(f"Euclidean: {euclidean_distance(get_embedding("photos/bartek-3.jpg"), get_embedding("photos/bartek-2.jpg"))}")

Cos: 0.5860831141471863
Euclidean: 0.9098536968231201
